# Complete AI/ML Pipeline with ONNX and Extended Features

Welcome to the **Complete AI/ML Pipeline** notebook! This notebook:

- Demonstrates a **production-ready** system for **document analysis** (summarization, topic extraction, translation, sentiment, Q&A, discussion generation, RAG, etc.)
- Includes **extended features** such as key ideas extraction, bullet-point summaries, chat interface, voice chat stubs, rewriting, recommendations, and summary refinement.
- Showcases how to **convert** models to ONNX format (via `convert_to_onnx.py`), load them dynamically, and fall back to standard PyTorch pipelines if ONNX models are not found.
- Provides a **Dockerfile** and a **FastAPI** server (`server.py`) for easy deployment.

This notebook is divided into the following sections:
1. **Setup and Requirements**
2. **Project Structure**
3. **convert_to_onnx.py** Script
4. **Core Code** (the `ai_ml` package)
5. **Extended Features**
6. **Dockerfile and requirements.txt**
7. **Optional: FastAPI server.py**
8. **Usage Instructions**

Let's begin!

## 1. Setup and Requirements

Before running this notebook or using the code, install the following:
```bash
pip install transformers torch langchain onnx onnxruntime python-dotenv fastapi uvicorn
```
Make sure your environment is properly set up (Python 3.9+ recommended).

## 2. Project Structure

Here is the recommended layout for this project:
```
.
├── convert_to_onnx.py
├── Dockerfile
├── requirements.txt
├── server.py
├── ai_ml/
│   ├── __init__.py
│   ├── config.py
│   ├── main.py
│   ├── backend.py
│   ├── models/
│   │   ├── __init__.py
│   │   ├── hf_model.py
│   │   ├── model_utils.py
│   │   └── onnx_helper.py
│   ├── processing/
│   │   ├── __init__.py
│   │   ├── summarizer.py
│   │   ├── translator.py
│   │   ├── topic_extractor.py
│   │   └── sentiment.py
│   ├── qa/
│   │   ├── __init__.py
│   │   └── qa_system.py
│   ├── discussion/
│   │   ├── __init__.py
│   │   └── discussion_generator.py
│   ├── rag/
│   │   ├── __init__.py
│   │   └── rag_system.py
│   └── extended_features/
│       ├── __init__.py
│       ├── key_ideas_extractor.py
│       ├── bullet_summary_generator.py
│       ├── chat_interface.py
│       ├── voice_chat.py
│       ├── rewriter.py
│       ├── recommendations_generator.py
│       └── refine_summary.py
```

In this notebook, we'll show all files. You can copy them into this structure. Then, you can optionally run the `convert_to_onnx.py` script if you want ONNX conversions.

## 3. `convert_to_onnx.py` Script

This script automates the conversion of all needed Hugging Face models to ONNX format. It uses the built-in transformers CLI. Adjust or remove what you don't need.


In [ ]:
%%writefile convert_to_onnx.py
import os
import subprocess
import sys

def run_conversion(model_name, feature, output_dir):
    """
    Runs the ONNX export command for a given model, feature, and output directory.
    """
    command = [
        sys.executable, "-m", "transformers.onnx",
        "--model", model_name,
        "--feature", feature,
        output_dir
    ]
    print("Running command:", " ".join(command))
    subprocess.run(command, check=True)

def main():
    # Create base directories for ONNX models
    base_dirs = [
        "onnx_models/summarizer",
        "onnx_models/qa",
        "onnx_models/discussion",
        "onnx_models/rag",
        "onnx_models/sentiment",
    ]
    
    # Translation model mappings (language code to model name)
    translations = {
        "fr": "Helsinki-NLP/opus-mt-en-fr",
        "de": "Helsinki-NLP/opus-mt-en-de",
        "es": "Helsinki-NLP/opus-mt-en-es",
        "it": "Helsinki-NLP/opus-mt-en-it",
        "zh": "Helsinki-NLP/opus-mt-en-zh",
    }
    
    for dir_path in base_dirs:
        os.makedirs(dir_path, exist_ok=True)
    
    # Create directories for translation models
    for lang in translations.keys():
        os.makedirs(f"onnx_models/translation/{lang}", exist_ok=True)
    
    try:
        # Convert Summarizer model
        run_conversion("facebook/bart-large-cnn", "summarization", "onnx_models/summarizer")
        
        # Convert QA model
        run_conversion("distilbert-base-cased-distilled-squad", "question-answering", "onnx_models/qa")
        
        # Convert Discussion Generator model (using GPT-2 for text generation)
        run_conversion("gpt2", "text-generation", "onnx_models/discussion")
        
        # Convert RAG model (for text2text-generation)
        run_conversion("facebook/rag-token-nq", "text2text-generation", "onnx_models/rag")
        
        # Convert Sentiment Analysis model
        run_conversion("distilbert-base-uncased-finetuned-sst-2-english", "sequence-classification", "onnx_models/sentiment")
        
        # Convert Translation models for each target language
        for lang, model in translations.items():
            run_conversion(model, "translation", f"onnx_models/translation/{lang}")
        
        print("All models have been successfully converted to ONNX format!")
    except subprocess.CalledProcessError as e:
        print("An error occurred during ONNX conversion:", e)
        sys.exit(1)

if __name__ == "__main__":
    main()


## 4. Core Code: `ai_ml` Package

We'll write out each file. In a real environment, place them under the `ai_ml/` directory as described.

In [ ]:
%%writefile ai_ml/__init__.py
# This file can remain empty or include package-wide initializations.
pass


In [ ]:
%%writefile ai_ml/config.py
# Model names for Hugging Face pipelines
MODEL_NAMES = {
    "summarizer": "facebook/bart-large-cnn",
    "qa": "distilbert-base-cased-distilled-squad",
    "discussion": "gpt2",
    "rag": "facebook/rag-token-nq",
    "topic_extractor": "facebook/bart-large-mnli"
    # DistilBERT sentiment model is loaded in hf_model.py
}

# Translation model mapping
TRANSLATION_MODELS = {
    "fr": "Helsinki-NLP/opus-mt-en-fr",
    "de": "Helsinki-NLP/opus-mt-en-de",
    "es": "Helsinki-NLP/opus-mt-en-es",
    "it": "Helsinki-NLP/opus-mt-en-it",
    "zh": "Helsinki-NLP/opus-mt-en-zh"
}

# Prompt templates for LangChain
PROMPT_TEMPLATES = {
    "summarization": "Summarize the following document in a concise manner:\n\n{text}\n\nSummary:",
    "qa": "Based on the context below, answer the question:\n\nContext: {context}\n\nQuestion: {question}\n\nAnswer:",
    "discussion": "Generate discussion points for the following document:\n\n{text}\n\nDiscussion Points:",
    "rag": "Analyze the following document and provide an in-depth analysis:\n\n{text}\n\nAnalysis:"
}


In [ ]:
%%writefile ai_ml/main.py
#!/usr/bin/env python
import argparse
import logging
import sys

from models.hf_model import load_models, load_translation_model
from processing.summarizer import summarize_text
from processing.topic_extractor import extract_topics
from processing.translator import translate_text
from processing.sentiment import analyze_sentiment
from qa.qa_system import answer_question
from discussion.discussion_generator import generate_discussion_points
from rag.rag_system import retrieval_augmented_generation

def setup_logging():
    logging.basicConfig(
        level=logging.INFO,
        format="%(asctime)s [%(levelname)s] %(name)s - %(message)s",
        handlers=[logging.StreamHandler(sys.stdout)]
    )

def main():
    setup_logging()
    logger = logging.getLogger(__name__)
    parser = argparse.ArgumentParser(
        description="Document Analysis App with LangChain Integration"
    )
    parser.add_argument("filepath", help="Path to the document file (txt) to analyze")
    parser.add_argument("--question", help="Question for Q&A", default=None)
    parser.add_argument(
        "--translate_lang",
        help="Target language code for translation (e.g., 'fr', 'de', 'es', 'it', 'zh')",
        default="fr"
    )
    args = parser.parse_args()

    try:
        with open(args.filepath, "r", encoding="utf-8") as f:
            document = f.read()
    except Exception as e:
        logger.error("Error reading file: %s", e)
        sys.exit(1)

    # Load LangChain chains and pipelines
    logger.info("Loading main models and chains...")
    models = load_models()

    # Dynamically load the translation pipeline for the specified target language
    try:
        translator = load_translation_model(args.translate_lang)
    except Exception as e:
        logger.error("Error loading translation model: %s", e)
        sys.exit(1)

    # Summarize the document (with chunking for long texts)
    try:
        summary = summarize_text(document, models["summarizer_chain"])
        print("=== Summary ===")
        print(summary)
    except Exception as e:
        logger.exception("Error during summarization: %s", e)

    # Extract key topics using the zero-shot classification pipeline
    try:
        topics = extract_topics(document, models["topic_extractor"])
        print("\n=== Key Topics ===")
        print(topics)
    except Exception as e:
        logger.exception("Error during topic extraction: %s", e)

    # Translate the document
    try:
        translation = translate_text(document, translator)
        print("\n=== Translated Document (to {}) ===".format(args.translate_lang))
        print(translation)
    except Exception as e:
        logger.exception("Error during translation: %s", e)

    # Sentiment analysis
    try:
        sentiment = analyze_sentiment(document, models["sentiment_analyzer"])
        print("\n=== Sentiment Analysis ===")
        print(sentiment)
    except Exception as e:
        logger.exception("Error during sentiment analysis: %s", e)

    # Q&A (if a question is provided)
    if args.question:
        try:
            answer = answer_question(document, args.question, models["qa_chain"])
            print("\n=== Q&A Answer ===")
            print(answer)
        except Exception as e:
            logger.exception("Error during Q&A: %s", e)

    # Generate discussion points
    try:
        discussion = generate_discussion_points(document, models["discussion_chain"])
        print("\n=== Discussion Points ===")
        print(discussion)
    except Exception as e:
        logger.exception("Error during discussion generation: %s", e)

    # Demonstrate RAG
    try:
        rag_answer = retrieval_augmented_generation(document, models["rag_chain"])
        print("\n=== RAG Generated Answer ===")
        print(rag_answer)
    except Exception as e:
        logger.exception("Error during RAG generation: %s", e)

if __name__ == "__main__":
    main()


In [ ]:
%%writefile ai_ml/backend.py
import logging
from models.hf_model import load_models, load_translation_model
from processing.summarizer import summarize_text
from processing.topic_extractor import extract_topics
from processing.translator import translate_text
from processing.sentiment import analyze_sentiment
from qa.qa_system import answer_question
from discussion.discussion_generator import generate_discussion_points
from rag.rag_system import retrieval_augmented_generation

logger = logging.getLogger(__name__)

# Cache models to avoid reloading on each call
MODELS = None

def initialize_models():
    global MODELS
    if MODELS is None:
        logger.info("Loading models and chains...")
        MODELS = load_models()
    return MODELS

def analyze_document(document: str, question: str = None, translate_lang: str = "fr"):
    """
    Analyze the provided document and return a dictionary with the results.

    Parameters:
      document (str): The document text.
      question (str): Optional question for Q&A.
      translate_lang (str): The target language code for translation.

    Returns:
      dict: Contains keys: summary, topics, translation, sentiment, qa, discussion, rag.
    """
    results = {}
    models = initialize_models()

    # Load translation pipeline for the specified target language.
    try:
        translator = load_translation_model(translate_lang)
    except Exception as e:
        logger.error("Error loading translation model: %s", e)
        translator = None

    # Summarization
    try:
        summary = summarize_text(document, models["summarizer_chain"])
        results["summary"] = summary
    except Exception as e:
        logger.exception("Error during summarization: %s", e)
        results["summary"] = None

    # Topic extraction
    try:
        topics = extract_topics(document, models["topic_extractor"])
        results["topics"] = topics
    except Exception as e:
        logger.exception("Error during topic extraction: %s", e)
        results["topics"] = None

    # Translation
    try:
        if translator:
            translation = translate_text(document, translator)
            results["translation"] = translation
        else:
            results["translation"] = None
    except Exception as e:
        logger.exception("Error during translation: %s", e)
        results["translation"] = None

    # Sentiment
    try:
        sentiment = analyze_sentiment(document, models["sentiment_analyzer"])
        results["sentiment"] = sentiment
    except Exception as e:
        logger.exception("Error during sentiment analysis: %s", e)
        results["sentiment"] = None

    # Q&A
    if question:
        try:
            answer = answer_question(document, question, models["qa_chain"])
            results["qa"] = answer
        except Exception as e:
            logger.exception("Error during Q&A: %s", e)
            results["qa"] = None
    else:
        results["qa"] = None

    # Discussion
    try:
        discussion = generate_discussion_points(document, models["discussion_chain"])
        results["discussion"] = discussion
    except Exception as e:
        logger.exception("Error during discussion generation: %s", e)
        results["discussion"] = None

    # RAG
    try:
        rag_answer = retrieval_augmented_generation(document, models["rag_chain"])
        results["rag"] = rag_answer
    except Exception as e:
        logger.exception("Error during RAG generation: %s", e)
        results["rag"] = None

    return results


### `ai_ml/models/`

In [ ]:
%%writefile ai_ml/models/__init__.py
from .hf_model import load_models, load_translation_model
from .model_utils import time_function, postprocess_text, ensemble_outputs, safe_execute
from .onnx_helper import check_onnx_model_exists, get_onnx_model_path


In [ ]:
%%writefile ai_ml/models/hf_model.py
import os
import logging
from transformers import pipeline, AutoTokenizer
from langchain.llms import HuggingFacePipeline
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

from ai_ml.config import MODEL_NAMES, TRANSLATION_MODELS, PROMPT_TEMPLATES
from .onnx_helper import check_onnx_model_exists, get_onnx_model_path

logger = logging.getLogger(__name__)

# Check if we should use ONNX models
USE_ONNX = os.getenv("USE_ONNX", "false").lower() == "true"

def load_models():
    models = {}
    
    # Summarizer chain
    try:
        logger.info("Loading summarizer pipeline...")
        summarizer_chain = _create_seq2seq_chain(
            model_name=MODEL_NAMES["summarizer"],
            onnx_subdir="summarizer",
            prompt_template=PROMPT_TEMPLATES["summarization"]
        )
        models["summarizer_chain"] = summarizer_chain
    except Exception as e:
        logger.exception("Error loading summarization chain: %s", e)
        raise e

    # QA chain
    try:
        logger.info("Loading QA pipeline...")
        qa_chain = _create_qa_chain(
            model_name=MODEL_NAMES["qa"],
            onnx_subdir="qa",
            prompt_template=PROMPT_TEMPLATES["qa"]
        )
        models["qa_chain"] = qa_chain
    except Exception as e:
        logger.exception("Error loading QA chain: %s", e)
        raise e

    # Discussion chain
    try:
        logger.info("Loading discussion generator pipeline...")
        discussion_chain = _create_textgen_chain(
            model_name=MODEL_NAMES["discussion"],
            onnx_subdir="discussion",
            prompt_template=PROMPT_TEMPLATES["discussion"]
        )
        models["discussion_chain"] = discussion_chain
    except Exception as e:
        logger.exception("Error loading discussion chain: %s", e)
        raise e

    # RAG chain
    try:
        logger.info("Loading RAG pipeline...")
        rag_chain = _create_text2text_chain(
            model_name=MODEL_NAMES["rag"],
            onnx_subdir="rag",
            prompt_template=PROMPT_TEMPLATES["rag"]
        )
        models["rag_chain"] = rag_chain
    except Exception as e:
        logger.exception("Error loading RAG chain: %s", e)
        raise e

    # Topic extractor
    try:
        logger.info("Loading topic extraction pipeline...")
        models["topic_extractor"] = pipeline("zero-shot-classification", model=MODEL_NAMES["topic_extractor"])
    except Exception as e:
        logger.exception("Error loading topic extractor: %s", e)
        raise e

    # Sentiment analyzer
    try:
        logger.info("Loading sentiment analyzer pipeline...")
        sentiment_model = "distilbert-base-uncased-finetuned-sst-2-english"
        if USE_ONNX:
            onnx_path = get_onnx_model_path("sentiment")
            if check_onnx_model_exists(onnx_path):
                tokenizer = AutoTokenizer.from_pretrained(sentiment_model)
                models["sentiment_analyzer"] = pipeline(
                    "sentiment-analysis",
                    model=onnx_path,
                    tokenizer=tokenizer,
                    framework="onnxruntime"
                )
            else:
                models["sentiment_analyzer"] = pipeline("sentiment-analysis", model=sentiment_model)
        else:
            models["sentiment_analyzer"] = pipeline("sentiment-analysis", model=sentiment_model)
    except Exception as e:
        logger.exception("Error loading sentiment analyzer: %s", e)
        raise e

    return models

def load_translation_model(target_lang):
    """
    Dynamically load a translation pipeline for the specified target language.
    """
    if target_lang not in TRANSLATION_MODELS:
        raise ValueError(f"Translation model for language '{target_lang}' is not configured.")
    
    model_name = TRANSLATION_MODELS[target_lang]
    task_name = f"translation_en_to_{target_lang}"
    logger.info("Loading translator model for language '%s'...", target_lang)

    if USE_ONNX:
        from transformers import AutoTokenizer
        onnx_path = get_onnx_model_path("translation", target_lang)
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        if check_onnx_model_exists(onnx_path):
            translator = pipeline(
                task_name,
                model=onnx_path,
                tokenizer=tokenizer,
                framework="onnxruntime"
            )
        else:
            translator = pipeline(task_name, model=model_name)
    else:
        translator = pipeline(task_name, model=model_name)
    return translator

def _create_seq2seq_chain(model_name, onnx_subdir, prompt_template):
    """
    Creates a seq2seq chain (e.g. summarization).
    """
    if USE_ONNX:
        onnx_path = get_onnx_model_path(onnx_subdir)
        if check_onnx_model_exists(onnx_path):
            tokenizer = AutoTokenizer.from_pretrained(model_name)
            pipe = pipeline("summarization", model=onnx_path, tokenizer=tokenizer, framework="onnxruntime")
        else:
            pipe = pipeline("summarization", model=model_name)
    else:
        pipe = pipeline("summarization", model=model_name)
    
    llm = HuggingFacePipeline(pipeline=pipe)
    template = PromptTemplate(input_variables=["text"], template=prompt_template)
    return LLMChain(llm=llm, prompt=template)

def _create_qa_chain(model_name, onnx_subdir, prompt_template):
    """
    Creates a QA chain.
    """
    if USE_ONNX:
        onnx_path = get_onnx_model_path(onnx_subdir)
        if check_onnx_model_exists(onnx_path):
            tokenizer = AutoTokenizer.from_pretrained(model_name)
            pipe = pipeline("question-answering", model=onnx_path, tokenizer=tokenizer, framework="onnxruntime")
        else:
            pipe = pipeline("question-answering", model=model_name)
    else:
        pipe = pipeline("question-answering", model=model_name)
    
    llm = HuggingFacePipeline(pipeline=pipe)
    template = PromptTemplate(input_variables=["context", "question"], template=prompt_template)
    return LLMChain(llm=llm, prompt=template)

def _create_textgen_chain(model_name, onnx_subdir, prompt_template):
    """
    Creates a text-generation chain (e.g. for discussion).
    """
    if USE_ONNX:
        onnx_path = get_onnx_model_path(onnx_subdir)
        if check_onnx_model_exists(onnx_path):
            tokenizer = AutoTokenizer.from_pretrained(model_name)
            pipe = pipeline("text-generation", model=onnx_path, tokenizer=tokenizer, framework="onnxruntime")
        else:
            pipe = pipeline("text-generation", model=model_name)
    else:
        pipe = pipeline("text-generation", model=model_name)
    
    llm = HuggingFacePipeline(pipeline=pipe)
    template = PromptTemplate(input_variables=["text"], template=prompt_template)
    return LLMChain(llm=llm, prompt=template)

def _create_text2text_chain(model_name, onnx_subdir, prompt_template):
    """
    Creates a text2text-generation chain (e.g. RAG).
    """
    if USE_ONNX:
        onnx_path = get_onnx_model_path(onnx_subdir)
        if check_onnx_model_exists(onnx_path):
            tokenizer = AutoTokenizer.from_pretrained(model_name)
            pipe = pipeline("text2text-generation", model=onnx_path, tokenizer=tokenizer, framework="onnxruntime")
        else:
            pipe = pipeline("text2text-generation", model=model_name)
    else:
        pipe = pipeline("text2text-generation", model=model_name)
    
    llm = HuggingFacePipeline(pipeline=pipe)
    template = PromptTemplate(input_variables=["text"], template=prompt_template)
    return LLMChain(llm=llm, prompt=template)


In [ ]:
%%writefile ai_ml/models/model_utils.py
import logging
import time
from typing import Callable, Any, Tuple

logger = logging.getLogger(__name__)

def time_function(func: Callable, *args, **kwargs) -> Tuple[Any, float]:
    """
    Times the execution of a function.
    Returns (result, elapsed_time).
    """
    start_time = time.time()
    result = func(*args, **kwargs)
    elapsed_time = time.time() - start_time
    logger.info("Function '%s' executed in %.2f seconds", func.__name__, elapsed_time)
    return result, elapsed_time

def postprocess_text(text: str) -> str:
    """
    Performs basic post-processing on text (trimming, ensuring period at end).
    """
    if text:
        text = " ".join(text.strip().split())
        if not text.endswith('.'):
            text += '.'
    return text

def ensemble_outputs(outputs: list, method: str = "first") -> str:
    """
    Combines outputs from multiple models. Currently supports "first" approach.
    """
    if not outputs:
        return ""
    if method == "first":
        return outputs[0]
    return outputs[0]

def safe_execute(func: Callable, *args, **kwargs) -> Tuple[Any, Exception]:
    """
    Safely executes a function and captures exceptions.
    Returns (result, error).
    """
    try:
        result = func(*args, **kwargs)
        return result, None
    except Exception as e:
        logger.exception("Error executing function '%s': %s", func.__name__, e)
        return None, e


In [ ]:
%%writefile ai_ml/models/onnx_helper.py
import os

def check_onnx_model_exists(model_path: str) -> bool:
    """
    Checks if a model.onnx file exists in the given directory.
    """
    onnx_file = os.path.join(model_path, "model.onnx")
    return os.path.exists(onnx_file)

def get_onnx_model_path(task: str, extra: str = "") -> str:
    """
    Constructs the path to an ONNX model for a given task.
    If extra is provided (e.g., a language code), it's appended.
    """
    base_dir = "onnx_models"
    if extra:
        return os.path.join(base_dir, task, extra)
    return os.path.join(base_dir, task)


### `ai_ml/processing/`

In [ ]:
%%writefile ai_ml/processing/__init__.py
from .summarizer import summarize_text
from .translator import translate_text
from .topic_extractor import extract_topics
from .sentiment import analyze_sentiment


In [ ]:
%%writefile ai_ml/processing/summarizer.py
def summarize_text(text, summarizer_chain, max_chunk_length=1000):
    """
    Summarizes the given text using the provided summarizer chain.
    If the text exceeds max_chunk_length, it is split into chunks.
    """
    if len(text) <= max_chunk_length:
        chunks = [text]
    else:
        chunks = [text[i:i+max_chunk_length] for i in range(0, len(text), max_chunk_length)]
    
    summaries = []
    for chunk in chunks:
        summary = summarizer_chain.run(text=chunk)
        summaries.append(summary.strip())
    return " ".join(summaries)


In [ ]:
%%writefile ai_ml/processing/translator.py
def translate_text(text, translator, max_length=512):
    """
    Translates the text using the provided translator pipeline.
    """
    if not translator:
        return ""
    translation_result = translator(text, max_length=max_length)
    return translation_result[0]["translation_text"]


In [ ]:
%%writefile ai_ml/processing/topic_extractor.py
def extract_topics(text, topic_extractor):
    """
    Extracts key topics from the text using zero-shot classification.
    """
    candidate_labels = [
        "technology", "health", "finance", "politics", 
        "sports", "education", "science", "entertainment"
    ]
    result = topic_extractor(text, candidate_labels)
    # Sort by score, return top three
    topics = sorted(zip(result["labels"], result["scores"]), key=lambda x: x[1], reverse=True)
    return [topic for topic, score in topics[:3]]


In [ ]:
%%writefile ai_ml/processing/sentiment.py
def analyze_sentiment(text, sentiment_analyzer):
    """
    Analyzes the sentiment of the text using the sentiment analysis pipeline.
    """
    result = sentiment_analyzer(text)
    return result[0]


### `ai_ml/qa/`

In [ ]:
%%writefile ai_ml/qa/__init__.py
from .qa_system import answer_question


In [ ]:
%%writefile ai_ml/qa/qa_system.py
def answer_question(context, question, qa_chain):
    """
    Answers the question based on the context using the QA chain.
    """
    answer = qa_chain.run(context=context, question=question)
    return answer.strip()


### `ai_ml/discussion/`

In [ ]:
%%writefile ai_ml/discussion/__init__.py
from .discussion_generator import generate_discussion_points


In [ ]:
%%writefile ai_ml/discussion/discussion_generator.py
def generate_discussion_points(text, discussion_chain):
    """
    Generates discussion points for the given text using a discussion chain.
    """
    result = discussion_chain.run(text=text)
    return result.strip()


### `ai_ml/rag/`

In [ ]:
%%writefile ai_ml/rag/__init__.py
from .rag_system import retrieval_augmented_generation


In [ ]:
%%writefile ai_ml/rag/rag_system.py
def retrieval_augmented_generation(text, rag_chain):
    """
    Demonstrates a retrieval-augmented generation approach (RAG).
    """
    result = rag_chain.run(text=text)
    return result.strip()


## 5. Extended Features

Below are the modules for key ideas, bullet-point summaries, chat interface, voice chat stubs, rewriting, recommendations, and summary refinement. Place them in `ai_ml/extended_features/`.

In [ ]:
%%writefile ai_ml/extended_features/__init__.py
# This file can remain empty or import extended feature functions.
pass


In [ ]:
%%writefile ai_ml/extended_features/key_ideas_extractor.py
import logging
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

logger = logging.getLogger(__name__)

KEY_IDEAS_PROMPT = """
Extract the main ideas (key takeaways) from the following document. Present them as a concise list:
Document:
{text}

Key Ideas:
"""

def generate_key_ideas(document: str, chain: LLMChain) -> str:
    """
    Generates the key ideas from the document using a specialized LLMChain.
    """
    try:
        template = PromptTemplate(input_variables=["text"], template=KEY_IDEAS_PROMPT)
        local_chain = LLMChain(llm=chain.llm, prompt=template)
        result = local_chain.run(text=document)
        return result.strip()
    except Exception as e:
        logger.exception("Error generating key ideas: %s", e)
        return ""


In [ ]:
%%writefile ai_ml/extended_features/bullet_summary_generator.py
import logging
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

logger = logging.getLogger(__name__)

BULLET_SUMMARY_PROMPT = """
Please summarize the following document in bullet points:
Document:
{text}

Bullet-Point Summary:
"""

def generate_bullet_summary(document: str, chain: LLMChain) -> str:
    """
    Generates a bullet-point summary of the document.
    """
    try:
        template = PromptTemplate(input_variables=["text"], template=BULLET_SUMMARY_PROMPT)
        local_chain = LLMChain(llm=chain.llm, prompt=template)
        result = local_chain.run(text=document)
        return result.strip()
    except Exception as e:
        logger.exception("Error generating bullet summary: %s", e)
        return ""


In [ ]:
%%writefile ai_ml/extended_features/chat_interface.py
import logging
from langchain.chains import ConversationChain

logger = logging.getLogger(__name__)

def chat_with_ai(user_input: str, conversation_chain: ConversationChain) -> str:
    """
    Allows a user to chat with the AI. The conversation_chain should maintain history.
    """
    try:
        response = conversation_chain.run(input=user_input)
        return response.strip()
    except Exception as e:
        logger.exception("Error in chat interface: %s", e)
        return "An error occurred."


In [ ]:
%%writefile ai_ml/extended_features/voice_chat.py
import logging

logger = logging.getLogger(__name__)

def voice_to_text(audio_data: bytes) -> str:
    """
    Convert the audio_data to text (STT). Stub implementation.
    """
    # e.g. use a whisper pipeline or an external service
    text = "STUB: recognized text from audio"
    return text

def text_to_voice(text: str) -> bytes:
    """
    Convert text to audio bytes (TTS). Stub implementation.
    """
    # e.g. use a TTS library or huggingface TTS pipeline
    audio_data = b"STUB: audio bytes"
    return audio_data

def voice_chat(audio_data: bytes, conversation_chain) -> bytes:
    """
    1. Convert user audio to text
    2. Pass text to conversation chain
    3. Convert AI response to audio
    """
    try:
        user_text = voice_to_text(audio_data)
        ai_response = conversation_chain.run(input=user_text)
        response_audio = text_to_voice(ai_response)
        return response_audio
    except Exception as e:
        logger.exception("Error in voice chat: %s", e)
        return b""


In [ ]:
%%writefile ai_ml/extended_features/rewriter.py
import logging
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

logger = logging.getLogger(__name__)

REWRITE_PROMPT = """
Rewrite the following text in a clear, coherent, and concise manner, preserving its meaning:
{text}

Rewritten:
"""

def rewrite_content(document: str, rewrite_chain: LLMChain) -> str:
    """
    Rewrites content using a text2text-generation or similar chain.
    """
    try:
        template = PromptTemplate(input_variables=["text"], template=REWRITE_PROMPT)
        local_chain = LLMChain(llm=rewrite_chain.llm, prompt=template)
        result = local_chain.run(text=document)
        return result.strip()
    except Exception as e:
        logger.exception("Error rewriting content: %s", e)
        return ""


In [ ]:
%%writefile ai_ml/extended_features/recommendations_generator.py
import logging
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

logger = logging.getLogger(__name__)

RECOMMENDATIONS_PROMPT = """
Read the document below and provide actionable recommendations or next steps:
Document:
{text}

Recommendations:
"""

def generate_recommendations(document: str, recommendation_chain: LLMChain) -> str:
    """
    Generates recommendations or next steps based on the document.
    """
    try:
        template = PromptTemplate(input_variables=["text"], template=RECOMMENDATIONS_PROMPT)
        local_chain = LLMChain(llm=recommendation_chain.llm, prompt=template)
        result = local_chain.run(text=document)
        return result.strip()
    except Exception as e:
        logger.exception("Error generating recommendations: %s", e)
        return ""


In [ ]:
%%writefile ai_ml/extended_features/refine_summary.py
import logging
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

logger = logging.getLogger(__name__)

REFINE_SUMMARY_PROMPT = """
We have an original document and an existing summary. Refine the summary to ensure it is accurate, clear, and concise:
Document:
{document}

Existing Summary:
{summary}

Refined Summary:
"""

def refine_summary(document: str, existing_summary: str, refine_chain: LLMChain) -> str:
    """
    Refines the existing summary by referencing the original document.
    """
    try:
        template = PromptTemplate(
            input_variables=["document", "summary"],
            template=REFINE_SUMMARY_PROMPT
        )
        local_chain = LLMChain(llm=refine_chain.llm, prompt=template)
        result = local_chain.run(document=document, summary=existing_summary)
        return result.strip()
    except Exception as e:
        logger.exception("Error refining summary: %s", e)
        return existing_summary


## 6. Dockerfile and requirements.txt

Below are the Dockerfile and requirements.txt to make the entire pipeline production-ready. Adjust them as needed for your environment.

In [ ]:
%%writefile Dockerfile
FROM python:3.9-slim

RUN apt-get update && apt-get install -y --no-install-recommends \
    build-essential \
    git \
 && rm -rf /var/lib/apt/lists/*

WORKDIR /app

COPY requirements.txt .
RUN pip install --no-cache-dir -r requirements.txt

COPY . .

EXPOSE 8000

# Default command (CLI). Adjust as needed.
CMD ["python", "ai_ml/main.py", "document.txt"]


In [ ]:
%%writefile requirements.txt
# Core ML libraries
torch>=1.10.0
transformers>=4.30.0
langchain>=0.0.220

# ONNX and runtime support
onnx>=1.12.0
onnxruntime>=1.14.0

# Optional: Hugging Face Optimum for enhanced ONNX support
optimum[onnxruntime]>=1.12.0

# For environment variable management (optional)
python-dotenv>=0.21.0

# FastAPI and Uvicorn for server.py example
fastapi>=0.95.0
uvicorn>=0.21.1


## 7. Optional: `server.py` (FastAPI)

If you want to expose an HTTP API, here’s a simple FastAPI server. It has one endpoint `/analyze` which calls `analyze_document` from `backend.py`. You can extend it to handle other endpoints for the extended features.

In [ ]:
%%writefile server.py
from fastapi import FastAPI, HTTPException
from pydantic import BaseModel
import uvicorn

from ai_ml.backend import analyze_document

app = FastAPI(title="Document Analysis API")

class AnalysisRequest(BaseModel):
    document: str
    question: str = None
    translate_lang: str = "fr"

@app.post("/analyze")
async def analyze(req: AnalysisRequest):
    try:
        results = analyze_document(
            document=req.document,
            question=req.question,
            translate_lang=req.translate_lang
        )
        return results
    except Exception as e:
        raise HTTPException(status_code=500, detail=str(e))

if __name__ == "__main__":
    uvicorn.run("server:app", host="0.0.0.0", port=8000, reload=True)


## 8. Usage Instructions

1. **Install Dependencies**
   ```bash
   pip install -r requirements.txt
   ```

2. **(Optional) Convert Models to ONNX**
   ```bash
   python convert_to_onnx.py
   ```
   Then set `USE_ONNX=true` in your environment if you want to use the ONNX versions.

3. **Run the CLI**
   ```bash
   python ai_ml/main.py document.txt --question "What is the main idea?" --translate_lang de
   ```

4. **Run the FastAPI Server** (optional)
   ```bash
   uvicorn server:app --host 0.0.0.0 --port 8000 --reload
   ```
   Then send a POST request to `http://localhost:8000/analyze` with JSON body:
   ```json
   {
       "document": "Your document text...",
       "question": "Some question...",
       "translate_lang": "fr"
   }
   ```

5. **Docker Build and Run** (optional)
   ```bash
   docker build -t document-analysis:latest .
   docker run -p 8000:8000 document-analysis:latest
   ```

6. **Extended Features**
   - Key ideas, bullet summaries, rewriting, recommendations, summary refinement, etc., can be integrated by loading or creating a dedicated `LLMChain` (like we do for summarization). Then call the relevant function from `ai_ml/extended_features/`. Example:
   ```python
   from ai_ml.extended_features.key_ideas_extractor import generate_key_ideas
   key_ideas = generate_key_ideas(document, my_key_ideas_chain)
   print(key_ideas)
   ```

Enjoy your **complete** AI/ML pipeline with ONNX support, extended features, Dockerization, and more!